# Environmental Impact of Agricultural Practices in the World

**ADA Project Milestone 2**

This notebook consists of our initial Data Analysis of the FAOSTAT dataset on Food an agriculture. We will first study the contents of the data and its strucuture, before restructuring it in order to start our analysis. Also, some research questions initially asked will be answered by the end of this notebook.

## A. Initial Analysis

The dataset initially contained 78 csv files, but some of them were discarded as they will not be useful for our analysis. We have selected 43 CSVs that would help us with our analysis.

In [18]:
from glob import glob
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
csv_files = glob('data/**/**.csv')
len(csv_files)

43

We split those 43 csv into different directories, one for each group of csv. Each group corresponds to one category:
```.
+-- data/
|   +-- emissions_agriculture/
|      +-- ...
|   +-- emissions_land/
|      +-- ...
|   +-- environment/
|      +-- ...
|   +-- forestry/
|      +-- ...
|   +-- inputs/
|      +-- ...
|   +-- population/
|      +-- ...
|   +-- production/
|      +-- ...
```

---
### A.1 Schema consistency
---

We will first study the schemas of all the csv files we have in order to see if they are consistent or require changes

#### 1. Checking column names across whole dataset
---

Now, let's scan all the csv files and check their schemas.

In [3]:
from data_processing import scan_columns
all_columns = scan_columns(csv_files)
print("The found columns, grouped, are:\n")
for cols, f in all_columns:
    print(list(sorted(cols)), f"Num files {len(f)}")

The found columns, grouped, are:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year', 'yearcode'] Num files 33
['country', 'countrycode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 4
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year', 'yearcode'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'note', 'unit', 'value', 'year', 'yearcode'] Num files 4
['country', 'countrycode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year', 'yearcode'] Num files 1


As we can see, sometimes the columns `area` and `areacode` are named `country` and `countrycode`, only because some csv files only contain country data, without country groups. We will rename those as to have a unified schema. Also, some files have the `note`, `elementgroup` and `months` columns. We will look into those in subsequent steps as we are now simply checking whether column naming is consistent.

In order to obtain a more consistent column naming, we will rename `country` to `area` and `countrycode` to `areacode`.

In [4]:
column_rename = {'country': 'area', 'countrycode': 'areacode'}

In [5]:
all_columns_2 = scan_columns(csv_files, column_rename)
print(f"After renaming, we obtain the following columns:\n")
for cols, f in all_columns_2:
    print(list(sorted(cols)), f"Num files {len(f)}")

After renaming, we obtain the following columns:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year', 'yearcode'] Num files 34
['area', 'areacode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 4
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year', 'yearcode'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'note', 'unit', 'value', 'year', 'yearcode'] Num files 4


#### 2. Checking which columns to drop
---

Now we have a few files that have different schemas. One column that we should look into before continuing is `note`, as it is in 4 files.

In [6]:
from data_processing import load_dataframe
def get_column_unique_values(files, col):
    # Function that returns the unique values of a column across multiple files
    values = []
    for f in files:
        df = load_dataframe(f, column_rename)
        vals = df[col].unique()
        values.append(vals)
    return values

files_with_note = all_columns_2[-1][1]
note_values = get_column_unique_values(files_with_note, 'note')
note_values

[array([nan]), array([nan]), array([nan]), array([nan])]

As we can see, all values for this column are NaN, so we can safely drop the column.

#### 3. Checking duplicate columns
---

We figured it would be useful to scan for duplicate columns in each dataframe (i.e. columns with different names but same values)

In [7]:
from data_processing import scan_column_duplicates
duplicates = scan_column_duplicates(csv_files, column_rename)
for c, f in duplicates:
    print(f"Duplicates for {c} in {len(f)} files")

Duplicates for [('yearcode', 'year')] in 39 files
Duplicates for [('elementgroup', 'elementcode')] in 3 files


As we can see, most files have `year` and `yearcode` columns which are equal. Hence, we can safely drop this column. However, for `elementgroup` and `elementcode`, they are equal in almost all CSV where they appear (3/4), but not all, so we cannot safely drop it without checking. We choose to keep `elementcode` when those two are equal, and keep them both when they are not.

Hence, we can define a list of columns to be dropped, but we need to check if they fulfill any of the following conditions:
 - NaN in all rows
 - Duplicate with another column

In [8]:
drop_columns = ["note", "yearcode", "elementgroup"]
all_columns_3 = scan_columns(csv_files, column_rename, drop_columns)
print(f"After renaming and dropping columns, we obtain the following columns:\n")
for cols, f in all_columns_3:
    print(list(sorted(cols)), f"Num files {len(f)}")

After renaming and dropping columns, we obtain the following columns:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 41
['area', 'areacode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year'] Num files 1


We now have 41 files with identical schemas, and 2 files that have a different one:
 - The file containing the `elementgroup` additional column 
 - The file with monthly data and no `item` and `itemcode` columns
 
To obtain the desired format, we can now call `load_dataframe(<file>, column_rename, drop_columns)` with `column_rename = {'country': 'area', 'countrycode': 'areacode'}` and `drop_columns = ["note", "yearcode", "elementgroup"]`

---
### A.2 Schema description

---
Now that we have a unified schema for (almost) all csv files, we can start looking into the meaning of each column and their possible value. 

#### 1. Area columns
---
We will first look into the columns `area` and `areacode`. According to FAOSTAT's website, each area is defined by a unique areacode, however some areas include other ones, i.e. there are grouped areas in the datasets. We would expect a one-to-one mapping between those two columns. Let's see how this looks like

To verify that it is indeed a one-to-one mapping, we will append all values from all csv files, and drop duplicates. Then we group by area and see if the length of the group is 1.

In [103]:
from data_processing import get_column_unique_values
from utils import is_unique_mapping

area_values = get_column_unique_values(csv_files, column_rename, drop_columns, ['area', 'areacode'])
is_unique_mapping(area_values, 'area', 'areacode')

False

It seems that the mapping is not one-to-one. Let's look into why that is. First we group by `areacode` and aggregate to a list of `area` and then we do the inverse. This way, we can see how the mapping is.

In [54]:
grpd = area_values.groupby('area')['areacode'].apply(list)
grpd[grpd.apply(lambda x: len(x) > 1)]

Series([], Name: areacode, dtype: object)

The mapping `area` -> `areacode` is unique, now for the opposite:

In [55]:
grpd = area_values.groupby('areacode')['area'].apply(list)
grpd[grpd.apply(lambda x: len(x) > 1)]

areacode
35                               [Cabo Verde, Cape Verde]
151     [Netherlands Antilles (former), Netherlands An...
154     [The former Yugoslav Republic of Macedonia, No...
167                             [Czechia, Czech Republic]
209                                 [Eswatini, Swaziland]
299           [Occupied Palestinian Territory, Palestine]
5000                             [World, World + (Total)]
5100                           [Africa, Africa + (Total)]
5101           [Eastern Africa, Eastern Africa + (Total)]
5102             [Middle Africa, Middle Africa + (Total)]
5103         [Northern Africa, Northern Africa + (Total)]
5104         [Southern Africa, Southern Africa + (Total)]
5105           [Western Africa, Western Africa + (Total)]
5200                       [Americas, Americas + (Total)]
5203       [Northern America, Northern America + (Total)]
5204         [Central America, Central America + (Total)]
5206                     [Caribbean, Caribbean + (Total)]
5207 

It seems that the same `areacode` is sometimes mapped to multiple `area`, most of the time by adding "+ (Total)". For that, we could drop the `area` column and keep working with `areacode`.

However, we still need a fixed database that maps the `areacode` to `area`, and for this, FAOSTAT provides one CSV file containing the mapping, as well as another one containing a description of country groups (i.e. "Asia") and which contries they contain.

In [106]:
area_codes = load_dataframe('data/countries.csv')
country_groups = load_dataframe('data/country_groups.csv')
country_groups.head()

,countrygroupcode,countrygroup,countrycode,country,m49code,iso2code,iso3code
0,5100,Africa,4,Algeria,12.0,DZ,DZA
1,5100,Africa,7,Angola,24.0,AO,AGO
2,5100,Africa,53,Benin,204.0,BJ,BEN
3,5100,Africa,20,Botswana,72.0,BW,BWA
4,5100,Africa,24,British Indian Ocean Territory,86.0,IO,IOT


Here, countries are grouped into multiple `countrygroup`, so we know exactly of which countries each group is formed. These country groups are present in the dataset as `area`, meaning there are aggregated values in the dataset. For example: we can find the emissions for "Algeria" and for "Africa", where the latter is an aggregated value over the whole group. We will need to be careful when aggregating values in the future, as we could account multiple times for one country.

In [111]:
area_codes.head()

,countrycode,country,m49code,iso2code,iso3code,startyear,endyear
0,2,Afghanistan,4.0,AF,AFG,NaN,NaN
1,5100,Africa,2.0,NaN,X06,NaN,NaN
2,284,Ã land Islands,248.0,NaN,ALA,NaN,NaN
3,3,Albania,8.0,AL,ALB,NaN,NaN
4,4,Algeria,12.0,DZ,DZA,NaN,NaN


This dataframe provides a mapping between `countrycode` and `country`, let's check if the mapping is unique, as well as all the values of `areacode` present in the dataset are in `area_codes`

In [115]:
is_unique_mapping(area_codes[['country', 'countrycode']].drop_duplicates(), 'country', 'countrycode')

True

In [122]:
all(x in area_codes.countrycode.unique() for x in area_values.areacode.unique())

True

Hence, we can use the dataframe `country_codes` to obtain the country's name from it's code. For this, we can now drop the `area` column in our dataset.

#### 2. Element Columns
---

The `element` and `elementcode` represent the measure quantity for a given `item`. A quantity has a name and a unit, which is why we believe these two columns should also have a one-to-one mapping accross the whole dataset. Also, since an `elementcode` potentially uniquely identifies (`element`, `unit`) pair, we might drop those two columns as to make the csv files smaller and easier to manipulate.

First let's check if indeed this mapping is one-to-one:

In [123]:
element_values = get_column_unique_values(csv_files, column_rename, drop_columns, ['elementcode', 'element', 'unit'])
is_unique_mapping(element_values, 'elementcode', ['element', 'unit'])

True

In [124]:
element_values.head()

,elementcode,element,unit
0,5111,Stocks,Head
171,5112,Stocks,1000 Head
684,5114,Stocks,No
0,5510,Production,tonnes
0,5313,Laying,1000 Head


As we can see, `elementcode` uniquely identify (`element`, `unit`) pairs, so we can safely drop those two columns and only use `elementcode`. We will later pivot each csv as to obtain all the `elementcode`s as columns, so we can reduce de number of rows significantly. A mapping using a dictionnary will of course be necessary in order to have a nice GUI where users can select the (element, unit) pair instead of the code.

#### 3. Item columns
---

According to FAOSTAT, the `item` and `itemcode` columns represent item on which measurements were done. For example an item can be `cattle` and the measurement can be "CH4 emissions in gigagrams". 
Similarly to what we did above, we expect `item` and `itemcode` to have a one-to-one relationship. Let's verify this using the same functions

In [136]:
item_values = get_column_unique_values(csv_files, column_rename, drop_columns, ['item', 'itemcode'], with_file=True)
is_unique_mapping(item_values[['item', 'itemcode']], 'item', 'itemcode')

False

It seems that `item` to `itemcode` is not unique for a few items, let's check those and try to understand why it is the case.

In [138]:
grpd = item_values.groupby('itemcode')['item'].agg(set)
grpd[grpd.apply(len) > 1]

Series([], Name: item, dtype: object)

The mapping `itemcode` -> `item` is unique, now let's check the other way around

In [139]:
grouped = item_values.groupby('item')['itemcode'].agg(set)
grouped[grouped.apply(len) > 1]

item
Ammonium nitrate (AN)                                  {1362, 4003}
Ammonium sulphate                                      {1361, 4002}
Burning - all categories                               {6795, 6798}
Cattle                                                  {866, 1757}
Chickens                                               {1057, 1054}
Cropland                                               {6620, 5070}
Disinfectants                                          {1358, 1351}
Forest land                                      {5065, 6749, 6646}
Grassland                                              {6794, 6983}
Mineral Oils                                           {1354, 1316}
Other Pesticides nes                                   {1355, 1359}
Other nitrogenous fertilizers, n.e.c.                  {4008, 1369}
Other potassic fertilizers, n.e.c.                     {4018, 1391}
Plant Growth Regulators                                {1356, 1341}
Potassium sulphate (sulphate of potash) (SO

Some items seem to have multiple (up to 3) different item codes, which doesn't seem very normal. 

**a.** It seems that some of those items correspond to nutrients provided throught fertilizers. Let's see in which files those appear. The items related to nutrients are the following :

In [140]:
nutrient_items = ["Ammonium nitrate (AN)", "Ammonium sulphate", "Other nitrogenous fertilizers, n.e.c.", "Other potassic fertilizers, n.e.c.", "Potassium sulphate (sulphate of potash) (SOP)", "Urea"]
item_values[item_values.item.isin(nutrient_items)].file.unique()

array(['data/inputs/Inputs_FertilizersArchive_E_All_Data_(Normalized).csv',
       'data/inputs/Inputs_FertilizersProduct_E_All_Data_(Normalized).csv'],
      dtype=object)

Those items are only present in two dataframes, related to Ferilizer use. If we look closely at the names of csv files, we can see that one of them is "Archive", 
while the other is product. After having looking and reading the documentation for this CSV [http://www.fao.org/faostat/en/#data/RA], we understand that it is an archive document, that has not been updated since 2002. 

Before that date, all data on Fertilizer was put into one single dataframe (i.e. info about production, trade and consumption) both in total nutrients and amount of product. After 2002, it was split into nutrient and product, which is why we have two `itemcode` values for those `items`. 

Hence, we should be careful when studying the fertilizer use, as data collection has changed in 2002.


**b.** Some items are related to pesticides. Let's in which csv files they appear

In [143]:
pesticide_items = ["Disinfectants", "Mineral Oils", "Other Pesticides nes", "Plant Growth Regulators"]
item_values[item_values.item.isin(pesticide_items)].file.unique()

array(['data/inputs/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv'],
      dtype=object)

Some duplicate items with the same name but different codes appear in only one csv file, which seems quite odd. We suspect having duplicate rows in that case. Let's check that out.

In [147]:
from data_processing import check_duplicate_items
t = load_dataframe('data/inputs/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv', column_rename, drop_columns)
check_duplicate_items(t, pesticide_items)

Duplicate item for Disinfectants codes [1358 1351]
Duplicate item for Mineral Oils codes [1354 1316]
Duplicate item for Other Pesticides nes codes [1359 1355]
Duplicate item for Plant Growth Regulators codes [1356 1341]


We now know that there are potentially duplicate rows for the same `item` but a different `itemcode` (with the same measurements). Hence, we need to add a functionnality that checks for duplicated items in the dataframe.

**c.** Now let's look at livestock items

In [180]:
livestock_items = ["Cattle", "Chickens"]
item_values[item_values.item.isin(livestock_items)].groupby(['item', 'itemcode']).agg(set)

file
item     itemcode                                                   
Cattle   866       {data/production/Production_Livestock_E_All_Da...
         1757      {data/emissions_agriculture/Emissions_Agricult...
Chickens 1054      {data/emissions_agriculture/Emissions_Agricult...
         1057      {data/production/Production_Livestock_E_All_Da...

The `itemcode` -> `item` seems one-to-one in each csv file, but we see that there are two distinct `itemcode` for "Cattle" and "Chickens" accross all csv files.

In [213]:
item_values[item_values.item.isin(livestock_items)][['file', 'itemcode', 'item']].drop_duplicates().values

array([['data/production/Production_Livestock_E_All_Data_(Normalized).csv',
        866, 'Cattle'],
       ['data/production/Production_Livestock_E_All_Data_(Normalized).csv',
        1057, 'Chickens'],
       ['data/environment/Environment_LivestockPatterns_E_All_Data_(Normalized).csv',
        866, 'Cattle'],
       ['data/environment/Environment_LivestockPatterns_E_All_Data_(Normalized).csv',
        1057, 'Chickens'],
       ['data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv',
        1757, 'Cattle'],
       ['data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv',
        1054, 'Chickens'],
       ['data/emissions_agriculture/Emissions_Agriculture_Enteric_Fermentation_E_All_Data_(Normalized).csv',
        1757, 'Cattle'],
       ['data/emissions_agriculture/Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).csv',
        1757, 'Cattle'],
       ['data/emissions_agriculture/Emissions_Agriculture_Manure_Management_E_All_Data_(N

It seems that different values for `itemcode` for both "Chickens" and "Cattles" differ when the measured quantity is realted to Manure. Thus, this might be a discrepancy in the data as it does not appear for other items. However, since the two different item codes appear in un-related csv files, we can keep them that way, and have two itemcodes that map to "Chickens" and "Cattle"

**d.** And finally, land related items

In [229]:
land_items = ["Burning - all categories", "Cropland", "Forest Land", "Grassland"]
item_values[item_values.item.isin(land_items)].sort_values(['item', 'file']).file.values

array(['data/emissions_agriculture/Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).csv',
       'data/emissions_land/Emissions_Land_Use_Burning_Biomass_E_All_Data_(Normalized).csv',
       'data/emissions_land/Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv',
       'data/environment/Environment_LandUse_E_All_Data_(Normalized).csv',
       'data/inputs/Inputs_LandUse_E_All_Data_(Normalized).csv',
       'data/emissions_agriculture/Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).csv',
       'data/emissions_land/Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv',
       'data/environment/Environment_LandCover_E_All_Data_(Normalized).csv'],
      dtype=object)

After looking into the description of each csv file, we have found the following explanations:
- For `Burning - all categories`:
    - for "Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).csv", it represents the emissions due to burning of Savanna (all categories) i.e. aggregated over all items
    - for "Emissions_Land_Use_Burning_Biomass_E_All_Data_(Normalized).csv", it represents the emissions due to the burning of Biomass.
- For `Grassland`: 
    - for "Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).csv", it represents the emissions burning of grassland
    - for "Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv" it represents the Green house gass emissions and removals from grasslands
    - for "Environment_LandCover_E_All_Data_(Normalized).csv" it represents the area of grassland
- For `Cropland`:
    - for "Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv" it represents the Green house gass emissions and removals from Croplands
    - for "Environment_LandUse_E_All_Data_(Normalized).csv" it represents the area of Cropland
    - for "Inputs_LandUse_E_All_Data_(Normalized).csv" it represents the usage of Cropland